In [1]:
from __future__ import absolute_import, division, print_function
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow_datasets as tfds
from tqdm import tqdm


import tensorflow as tf
import tensorflow_hub as hub

In [2]:
module_selection = ("mobilenet_v2", 224, 1280)
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


In [3]:
splits = ("train[:80]", "train[:10]", "train[:10]")
splits, info = tfds.load('cats_vs_dogs',with_info=True, as_supervised=True, split = splits)

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [4]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return image, label

In [5]:
BATCH_SIZE = 32

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_examples = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([32, 224, 224, 3])

In [8]:
do_fine_tuning=False

In [9]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=do_fine_tuning)

In [10]:
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([feature_extractor,
                            tf.keras.layers.Dense(num_classes, activation='softmax')])
model.summary()

Building model with https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [11]:
NUM_LAYERS = 10
if do_fine_tuning:
    feature_extractor.trainable=True
    
    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable=True
else:
    feature_extractor.trainable=False

In [13]:
if do_fine_tuning:
    model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.002, momentum=0.9),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
else:
    model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [14]:
EPOCHS = 5
hist = model.fit(train_batches,
                epochs = EPOCHS,
                validation_data= validation_batches)

Epoch 1/5
3/3 [==============================] - 4s 1s/step - loss: 0.7379 - accuracy: 0.6125 - val_loss: 0.5886 - val_accuracy: 0.8000
Epoch 2/5
3/3 [==============================] - 1s 411ms/step - loss: 0.4316 - accuracy: 0.8250 - val_loss: 0.3812 - val_accuracy: 0.8000
Epoch 3/5
3/3 [==============================] - 1s 411ms/step - loss: 0.2540 - accuracy: 0.9000 - val_loss: 0.2570 - val_accuracy: 0.8000
Epoch 4/5
3/3 [==============================] - 1s 420ms/step - loss: 0.1490 - accuracy: 0.9625 - val_loss: 0.1798 - val_accuracy: 1.0000
Epoch 5/5
3/3 [==============================] - 1s 421ms/step - loss: 0.0939 - accuracy: 1.0000 - val_loss: 0.1324 - val_accuracy: 1.0000


In [15]:
CATS_VS_DOGS_SAVED_MODEL = 'exp_saved_model'

In [16]:
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: exp_saved_model/assets


INFO:tensorflow:Assets written to: exp_saved_model/assets


In [18]:
%%bash -s $CATS_VS_DOGS_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [19]:
loaded = tf.saved_model.load(CATS_VS_DOGS_SAVED_MODEL)

In [24]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures['serving_default']
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'keras_layer_input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_layer_input')})
{'dense': TensorSpec(shape=(None, 2), dtype=tf.float32, name='dense')}


In [25]:
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)